In [3]:
#1
%matplotlib inline
import pandas as pd
import csv
import numpy as np
import time 
import datetime
import matplotlib.pyplot as plt
import pylab as pl

In [8]:
#2 creating input dataframe
max_distance = 80
MaxPts = 10
dayRadius = 10

df = pd.DataFrame.from_csv('D:/Nanohub/results/data extract from database/in_STDB_2015_jan_dec.csv', sep ='\t', encoding = 'UTF-8', header = None)
df.columns = ['ip','resource_id','datetime','latitude','longitude','child_id','title', 'ip_domainname', 'ip_country', 'uidNumber']
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(by = 'resource_id', axis = 0, ascending = True, na_position ='first')
df_no_dups = df.drop_duplicates(subset = ['ip','resource_id'], keep = 'first')
doy = []
for date in df['datetime']:
    doy.append(date.dayofyear)
df['DoY'] = doy

df_10 = df[df.groupby('resource_id').cumcount(ascending=False) >= 10]
distinct_resources = list(df_10['resource_id'].unique())


In [10]:
#3 comipling ST_DBSCAN

def ST_DBSCAN(points,max_distance,MinPts,dayRadius):
    global visited 
    visited = []
    noise = []
    c_neighbors = []
    cluster_id = 0
    clusters = []
    in_cluster = []
    for p in points: 
        if p not in visited:
            # neighbor_points = []
            visited.append(p)
            NeighborPts = regionQuery(p,points,max_distance,dayRadius)
            if len(NeighborPts) < MinPts:
                noise.append(p)
            else:
                cluster_id = cluster_id + 1
                g = expandCluster(p,NeighborPts,max_distance,MinPts,in_cluster,cluster_id)
                clusters.append(g)
                c_neighbors=NeighborPts+[cluster_id]
                #print c_neighbors
                c_neighbors.append(g)
    return clusters, c_neighbors

#return len(NeighborPts)

def expandCluster(p,NeighborPts,max_distance,MinPts,in_cluster,cluster_id):
    in_cluster.append(p)
    cluster = []
    t_p = list(p) + [cluster_id]
    cluster.append(t_p)
    for point in NeighborPts:
        if point not in visited:
            visited.append(point)
            new_neighbors = regionQuery(point,points,max_distance,dayRadius)
            if len(new_neighbors) >= MinPts: 
                new_neighbors.append(NeighborPts)
            if point not in in_cluster:
                 t_point = list(point) + [cluster_id]
                 in_cluster.append(point)
                 cluster.append(t_point)
    return cluster


def regionQuery(p,points,max_distance,dayRadius):
    neighbor_points = []
    loc1 = (p[0],p[1])
    for j in points:
        if j != p:
            # print 'P is %s and j is %s' % (p[0],j[0])
            if abs(p[2]-j[2]) < dayRadius:
                loc2 = (j[0],j[1])
                dist = haversine(loc1, loc2)
                if dist <= max_distance:
                    neighbor_points.append(j)
    neighbor_points.append(p) 
    return neighbor_points   


In [ ]:

#4 STDBSCAN call
for l in distinct_resources: #range(21095,22000):
    df_filtered = df_10[df_10['resource_id'] == l]
    df_filtered =df_filtered[df_filtered['latitude'] > 0]
    tups = df_filtered.set_index(['latitude'])[['longitude','DoY','resource_id','ip']].to_records().tolist()
    points = tups
    core, neigh = ST_DBSCAN(points,max_distance,MaxPts,dayRadius)
    
    fname = 'D:/Nanohub/results/GIS6/' + str(l) +'.csv'
    
    f = open(fname, 'wt')
    try:
        writer = csv.writer(f, lineterminator = '\n')
        #writer.writerow( ('Latitude', 'Longitude', 'DoY' , 'res_id' , 'cluster_id') )

        for j in range(len(core)):
            for i in range(len(core[j])):
                    writer.writerow( core[j][i] )
    finally:
        f.close()